# National Leagues Data

In [103]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json

In [104]:
def url_league(league_name, season):
    return f"https://understat.com/league/{league_name}/{season}"

In [105]:
def get_data(league_name, season, i):
    # get the url
    url=url_league(league_name, season)

    # parse the html page
    dataLigue1=requests.get(url_league("Ligue_1","2017"), "html.parser")
    soupLigue1=BeautifulSoup(dataLigue1.text, "html.parser")

    # get the scripts
    scripts=soupLigue1.find_all('script')
    strings=scripts[i].string

    # Convert as JSON
    ind_start = strings.index("('")+2 
    ind_end = strings.index("')") 
    json_data = strings[ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

    # Convert as DataFrame
    df=pd.read_json(json_data)

    return df

In [ ]:
# save the data
df_players = get_data("ligue 1", "2017", 2)
df_players.to_csv("all_player_big_6.csv")

# Scrap wiki

In [7]:
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd

    from time import sleep
    from selenium import webdriver
    #from selenium.webdriver.common.keys import Keys
    #import json
    from tqdm import tqdm, tqdm_pandas
    tqdm_pandas(tqdm())
    import time 
    from time import sleep

0it [00:00, ?it/s]


In [ ]:
#load the data

all_player_big_6 = pd.read_csv('all_player_big_6.csv')


In [ ]:
# take a dico of the wiki data
# return a row with the data
def get_out_of_the_dico(dico,player_id):
    player_international = None 
    actual_team = None 
    number = None 
    Name = None 
    mois_de_naissance = None 
    Nationalite = None 
    jour_de_naissance = None 
    anne_de_naissance = None 
    age_actuel = None 
    taille_en_m = None 
    Poste = None 
    Pied_fort = None 
    if "Sélections" in dico.keys() :
        player_international = dico["Sélections"]
    if "Équipe" in dico.keys():
        actual_team = dico["Équipe"]
    if "Numéro"in dico.keys():
        number = dico["Numéro"]
    if "Nom" in dico.keys():
        Name = dico["Nom"]
    if "Nationalité"in dico.keys():
        Nationalite = dico["Nationalité"]
    if "Naissance" in dico.keys():
        mois_de_naissance = dico["Naissance"].split()[1]
        jour_de_naissance = dico["Naissance"].split()[0]
        anne_de_naissance = dico["Naissance"].split()[2]
        age_actuel = dico["Naissance"].split()[3][1:]
    if "Taille" in dico.keys():
        taille_en_m = dico["Taille"].split()[0]
    if "Poste" in dico.keys():
        Poste = dico['Poste']
    if "Pied fort" in dico.keys():
        Pied_fort = dico['Pied fort']
    return player_international, actual_team,number,Name,Nationalite,mois_de_naissance,jour_de_naissance,anne_de_naissance,age_actuel,taille_en_m,Poste,Pied_fort,player_id

In [ ]:
# get the urlpage
# return dico with the data
def get_the_data_contains_on_the_page(htmlplayerpage):
    stopwords = ['']
    dico = {}
    dico['Sélections']=False
    # get all the data that we want 
    for item in htmlplayerpage.find("table",{"class":"infobox_v2 noarchive"}).findAll('tr'):
        text_item = item.getText()
        if text_item=='\nSélections en équipe nationale2':
            dico['Sélections']=True
        elif text_item!=None:
            text_item = text_item.split('\n')
            text_item = [word for word in text_item if word not in stopwords]
            if len(text_item)==2:
                dico[text_item[0]]=text_item[1]
    return dico 

In [ ]:
# get a  player name
# return the data of this player after scarping wiki
def get_the_htlm_page_of_a_player(row):
    player_name = row["player_name"]
    id = row["id"]
    player_name_send = player_name
    base_url = "https://fr.wikipedia.org/wiki/"
    player_name = player_name.split()
    # get the name looks like what ask wiki 
    if len(player_name)==2:
        url = base_url+ player_name[0]+'_'+player_name[1]
    else:
        url = base_url+ player_name[0]
    # get the url data 
    request_data =requests.get(url)
    htmlplayerpage=BeautifulSoup(request_data.text, "html.parser")
    # try if that is the good url 
    try: 
        htmlplayerpage.find("table",{"class":"infobox_v2 noarchive"}).findAll('tr')
    except:
        return None
    else:
        # work on the data 
        dico =get_the_data_contains_on_the_page(htmlplayerpage)
        data = get_out_of_the_dico(dico,id)
        return data
    

In [ ]:
data_scrap = all_player_big_6.progress_apply(get_the_htlm_page_of_a_player,axis=1)
filtered_list = [x for x in data_scrap if x is not None]

100%|██████████| 540/540 [01:39<00:00,  5.45it/s]


In [3]:
# create a df for the new data
def new_data(filtered_list,all_player_big_6):
    wiki_data= pd.DataFrame(columns=["player_international", "actual_team","number","Name","Nationalite","mois_de_naissance","jour_de_naissance","anne_de_naissance","age_actuel","taille_en_m","Poste","Pied_fort","id"])
    for i in filtered_list:
        wiki_data.loc[len(wiki_data)]=i
    final_data_set = wiki_data.merge(all_player_big_6,on= "id")
    return final_data_set

In [ ]:
# save the new df
final_data_set = new_data(filtered_list,all_player_big_6)
final_data_set.to_csv("df_player_wiki.csv")

NameError: name 'final_data_set' is not defined

In [ ]:
from selenium import webdriver
driver =  webdriver.Firefox()

In [ ]:
driver.get("https://www.fifaindex.com/")

# Scrap index fifa

In [ ]:
#get url player page
#return dico with all the data
def return_a_dico_with_all_the_data(url_player):
    dico = {}
    request_data =requests.get(url_player)
    htmlplayerpage=BeautifulSoup(request_data.text, "html.parser")
    # get all the data we interst of 
    for item in htmlplayerpage.find("div",{"class":"masonry"}).findAll('p'):
        text_item = item.getText()
        if text_item!=None:
            text_item = text_item.split()
            if len(text_item)==2:
                dico[text_item[0]]=text_item[1]
    return dico

In [ ]:
#player name and bool if this is the first time
#Return player name for the scaping
def get_correct_player_name_for_selenium(player_name_df,first):
    player_name_df_splip = player_name_df.split()
    if len(player_name_df_splip)>=1 and first==True:
        if len(player_name_df_splip)==2:
            return player_name_df
        else:
            return player_name_df_splip[-1]
    else:
        return player_name_df_splip[-1]

In [ ]:
#get a player name
# return all the data
def get_the_url_player(row):
    dico ={}
    # get a player like the website ask
    player_name = get_correct_player_name_for_selenium(row['player_name'],True)
    # find the bar
    shearch_bar = driver.find_element('xpath', '//*[@id="site-search"]')
    shearch_bar.click()
    # rigth a player name 
    shearch_bar.send_keys(player_name)
    sleep(1.1)
    # try to get the data 
    try:
        driver.find_element("xpath",'/html/body/header/nav/div/form/span/div/div/div[1]/div')
    except:
        # try again with a other name 
        driver.get("https://www.fifaindex.com/")
        player_name = get_correct_player_name_for_selenium(row['player_name'],False)
        shearch_bar = driver.find_element('xpath', '//*[@id="site-search"]')
        shearch_bar.click()
        shearch_bar.send_keys(player_name)
        sleep(2)
        try: 
            driver.find_element("xpath",'/html/body/header/nav/div/form/span/div/div/div[1]/div')
        except:
            return None
        else:
            first_player = driver.find_element("xpath",'/html/body/header/nav/div/form/span/div/div/div[1]/div')
            first_player.click()
            sleep(0.2)
            url_player = driver.current_url
            dico = return_a_dico_with_all_the_data(url_player)
            dico['id'] = row['id']
    else:
        # get the url of the player 
        first_player = driver.find_element("xpath",'/html/body/header/nav/div/form/span/div/div/div[1]/div')
        first_player.click()
        sleep(0.2)
        url_player = driver.current_url
        dico = return_a_dico_with_all_the_data(url_player)
    return dico 


In [ ]:
df = pd.read_csv('df_player_wiki_fifa.csv')

In [ ]:
for index,row in tqdm(df.iterrows()):
    if type(row["dico_fifa"])==float:
        row["dico_fifa"] = get_the_url_player(row)
        df.at[index] = row

72it [08:46,  7.31s/it]


In [148]:
df.to_csv('df_player_wiki_fifa.csv')

In [150]:
df_with_new_col = pd.DataFrame(columns=["Dribbling","Marking","Aggression" ,"Reactions" ,"Interceptions" ,"Vision" ,"Composure" ,"Crossing" ,"Acceleration" ,"Strength" ,"Balance" ,"Agility","Penalties" ,"Volleys" ,"Power" ,"Jumping" ,"Heading","Curve","Injury","id","Finishing"])

In [151]:
# get a dico 
# return the data 
def get_out_of_the_dico(dico,player_id):
    if type(dico)!= float: 
        try:
            ast.literal_eval(str(dico))
        except:
            return None
        else:
            dico = ast.literal_eval(str(dico))
            Dribbling = None 
            Marking = None 
            Aggression = None 
            Reactions = None 
            Interceptions = None 
            Vision = None 
            Composure = None 
            Crossing = None 
            Acceleration = None 
            Strength = None 
            Balance = None 
            Agility = None
            Penalties = None 
            Volleys = None 
            Power = None 
            Jumping = None 
            Heading = None 
            Curve = None
            Injury = None 
            Finishing = None
            if "Dribbling" in dico.keys() :
                Dribbling = dico["Dribbling"]
            if "Marking" in dico.keys():
                Marking = dico["Marking"]
            if "Aggression"in dico.keys():
                Aggression = dico["Aggression"]
            if "Reactions" in dico.keys():
                Reactions = dico["Reactions"]
            if "Interceptions"in dico.keys():
                Interceptions = dico["Interceptions"]
            if "Vision" in dico.keys():
                Vision = dico["Vision"]
            if "Composure" in dico.keys():
                Composure = dico["Composure"].split()[0]
            if "Crossing" in dico.keys():
                Crossing = dico['Crossing']
            if "Acceleration" in dico.keys():
                Acceleration = dico['Acceleration']
            if "Strength" in dico.keys():
                Strength = dico["Strength"].split()[0]
            if "Agility" in dico.keys():
                Agility = dico['Agility']
            if "Penalties" in dico.keys():
                Penalties = dico['Penalties']
            if "Volleys" in dico.keys():
                Volleys = dico["Volleys"].split()[0]
            if "Power" in dico.keys():
                Power = dico['Power']
            if "Jumping" in dico.keys():
                Jumping = dico['Jumping']
            if "Heading" in dico.keys():
                Heading = dico["Heading"].split()[0]
            if "Curve" in dico.keys():
                Curve = dico['Curve']
            if "Injury" in dico.keys():
                Injury = dico['Injury']
            if "Finishing" in dico.keys():
                Finishing = dico['Finishing']
            return Dribbling,Marking,Aggression ,Reactions ,Interceptions ,Vision ,Composure ,Crossing ,Acceleration ,Strength ,Balance ,Agility,Penalties ,Volleys ,Power ,Jumping ,Heading,Curve,Injury,player_id,Finishing
    return None

In [152]:
for index,row in df.iterrows():
    data = get_out_of_the_dico(row["dico_fifa"],row["id"])
    df_with_new_col.loc[len(df_with_new_col)]= data
    

C:\Users\regis\AppData\Local\Temp\ipykernel_13632\3026797825.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_with_new_col.loc[len(df_with_new_col)]= data


In [154]:
final_df = df.merge(df_with_new_col,on='id')

In [157]:
final_df.to_csv('joueur_ligue_1_stat_2017_prix_et_stat_fifa.csv')

# Scrap marquet

In [ ]:
#get the player name
#return value
def get_the_value(player_name):
    #get the name of the player 
    player_name = get_correct_player_name_for_selenium(row['player_name'],True)
    # rigth the name in the shearch_bar
    shearch_bar = driver.find_element('xpath', '//*[@id="site-search"]')
    shearch_bar.send_keys(player_name)
    lunch = driver.find_element('xpath', '//*[@id="site-search"][2]')
    # do the reschear 
    lunch.click()
    player_value = driver.find_element('xpath', '/html/body/div[2]/main/div[1]/div[2]/div/div/div[1]/a/div[1]/img')
    # get the data 
    data = player_value.getattribute("Text")
    return data

In [ ]:
final_df["value_fifa"]=None
final_df["value_fifa"] = final_df["value_fifa"].progress_apply(get_the_value)

In [ ]:
final_df.to_csv('all_data.csv')